In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta

import ast
import string
import re

import nltk
nltk.download('punkt')

import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
def normalize_reviews(filepath, filename):
  '''
  1. Drop duplicated rows
  2. Normalize dates to only the year
  3. Normalize ratings to 10
  '''
  
  df = pd.read_excel(filepath)
  
  # Drop duplicated rows
  df.drop_duplicates(keep='first', inplace=True)
  
  # 1. Normalize dates to only year
  
  # For Google Review dates
  # fix_date_week | fix_date_month | fix_date_year
  # Takes current date and minus off respective day, week, month or year
  def fix_date_day(day):
    date = datetime.datetime.now()
    date = date.date()
    newdate = date - relativedelta(days=day)
    return newdate
  
  def fix_date_week(week):
    date = datetime.datetime.now()
    date = date.date()
    newdate = date - relativedelta(weeks=week)
    return newdate
  
  def fix_date_month(month):
    date = datetime.datetime.now()
    date = date.date()
    newdate = date - relativedelta(months=month)
    return newdate
  
  def fix_date_year(year):
    date = datetime.datetime.now()
    date = date.date()
    newdate = date - relativedelta(years=year)
    return newdate
  
  # Replace date strings with proper year
  # Only up to 10 years
  replace_date = {'sehari lalu': fix_date_day(1),
                  '2 hari lalu': fix_date_day(2),
                  '3 hari lalu': fix_date_day(3),
                  '4 hari lalu': fix_date_day(4),
                  '5 hari lalu': fix_date_day(5),
                  '6 hari lalu': fix_date_day(6),
                  'seminggu lalu': fix_date_week(1),
                  '2 minggu lalu': fix_date_week(2),
                  '3 minggu lalu': fix_date_week(3),
                  '4 minggu lalu': fix_date_week(4),
                  'sebulan lalu': fix_date_month(1),
                  '2 bulan lalu': fix_date_month(2),
                  '3 bulan lalu': fix_date_month(3),
                  '4 bulan lalu': fix_date_month(4),
                  '5 bulan lalu': fix_date_month(5),
                  '6 bulan lalu': fix_date_month(6),
                  '7 bulan lalu': fix_date_month(7),
                  '8 bulan lalu': fix_date_month(8),
                  '9 bulan lalu': fix_date_month(9),
                  '10 bulan lalu': fix_date_month(10),
                  '11 bulan lalu': fix_date_month(11),
                  'setahun lalu': fix_date_year(1),
                  '2 tahun lalu': fix_date_year(2),
                  '3 tahun lalu': fix_date_year(3),
                  '4 tahun lalu': fix_date_year(4),
                  '5 tahun lalu': fix_date_year(5),
                  '6 tahun lalu': fix_date_year(6),
                  '7 tahun lalu': fix_date_year(7),
                  '8 tahun lalu': fix_date_year(8),
                  '9 tahun lalu': fix_date_year(9),
                  '10 tahun lalu': fix_date_year(10),}

  
  # Google review mask
  dfg = df[(df['source']=='google_reviews')].copy()
  dfg = dfg.replace({"date": replace_date})
  dfg['date'] = pd.DatetimeIndex(dfg['date']).year
  
  # Other websites
  # Create mask for non-google rows
  ngmask = (df['source'] != 'google_reviews')

  # Dataframe for non-google rows
  df_ng = df[ngmask]

  # Get proper date rows from non-google df to separate dataframe
  df_ng_prop = df_ng[df_ng['date'].apply(lambda x: isinstance(x, datetime.date))].copy()

  # Extract only year from proper date rows
  df_ng_prop['date'] = pd.DatetimeIndex(df_ng_prop['date']).year

  # Get improper date rows from non-google df to separate dataframe
  df_ng_improp = df_ng[df_ng['date'].apply(lambda x: not isinstance(x, datetime.date))].copy()

  # Convert datatype to string and only extract last 4 index for year
  df_ng_improp['date'] = df_ng_improp['date'].astype(str).str[-4:]

  # Rejoin all dataframes
  df = pd.concat([dfg, df_ng_prop, df_ng_improp], ignore_index=True)
  df['date'] = df['date'].astype(int)
  
  #2. Normalize ratings to 10
  
  # Normalize klook ratings
  replace_rating = {4:8,
                    'Baik': 8,
                    'Sangat Direkomendasikan': 10}
  df[(df['source'] == 'klook')] = df.replace({'rating': replace_rating})
  
  # Normalize TripAdvisor ratings
  ta_mask = (df['source'] == 'tripadvisor')
  # Retrieve index 0 string of rating and multiply by 2
  df.loc[ta_mask, 'rating'] = (df.loc[ta_mask, 'rating'].str[0].astype(int))*2
  
  # Normalize Google Review ratings
    # Mask for Google Review ratings
  g_mask = (df['source'] == 'google_reviews')
  # Retrieve index 14 string of rating and multiply by 2
  df.loc[g_mask, 'rating'] = (df.loc[g_mask, 'rating'].str[14].astype(int))*2
  
  return df

In [4]:
# Enter input filepath
df = normalize_reviews("/content/gdrive/MyDrive/data/raw/attractions_master.xlsx", "attractions_master.xlsx")
df.head()

,user,date,rating,review,page_url,date_scraped,source,attraction
0,desuka Panjaitan,2021,10,"Belum Pernah Masuk kedalam, hanya diluar saja....",https://www.google.com/search?q=universal+stud...,2022-04-27,google_reviews,uss
1,Riska Septi Damayanti,2021,10,Sumpahh kalo ke Singapore wajib banget ke sini...,https://www.google.com/search?q=universal+stud...,2022-04-27,google_reviews,uss
2,Mochamad Naufal Irfansyah,2021,10,"Seru bisa jajan coklat enak dan murah, kalo ma...",https://www.google.com/search?q=universal+stud...,2022-04-27,google_reviews,uss
3,T1y25,2022,10,"Luas banget,banyak spot fotonya,wahananya keren²",https://www.google.com/search?q=universal+stud...,2022-04-27,google_reviews,uss
4,Rengga Utami,2021,10,Wahana Sangat Lengkap ₩Pokoknya Tempatnya Luar...,https://www.google.com/search?q=universal+stud...,2022-04-27,google_reviews,uss


In [5]:
def preprocess_reviews(df):
  '''
  1. Remove website URLs and non-characters
  2. Remove all characters except alphabets, numbers and punctuations
  3. Replace the asterisks people use in place of swear words with a empty string ''
  4. Remove leftover emojis
  5. If uppercase letter exists, add in _ before uppercase letter and replace uppercase letter to lowercase letter
  6. Replace slang words
  7. Change the character after _ back to uppercase
  8. Remove empty rows in case review only contains emojis
  9. Drop rows that have reviews from year 2017and earlier
  '''
  file = open("/content/gdrive/MyDrive/data/bahasa_indonesia_slangwords.txt", "r")
  contents = file.read()
  slangwords = ast.literal_eval(contents)
  
  def basic_cleaning(text):
      text=re.sub(r'https?://www\.\S+\.com','', text) # Remove website URLs and non-characters
      text=re.sub(r'[^A-Za-z0-9`~!@#%&-_=;:,<.>/\.\+\*\?\^\$\(\)\[\]\{\}\|\\|\s]','', text) # Remove all characters except alphabets, numbers and punctuations
      text=re.sub(r'\*+','', text) # Replace the asterisks people use in place of swear words with a empty string ''
      return text
  
  def remove_emoji(text):
      emoji_pattern = re.compile(pattern = "["
          u"\U0001F600-\U0001F64F"  # emoticons
          u"\U0001F300-\U0001F5FF"  # symbols & pictographs
          u"\U0001F680-\U0001F6FF"  # transport & map symbols
          u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
          u"\U00002702-\U000027B0"  
          u"\U000024C2-\U0001F251"
                           "]+", flags = re.UNICODE)
      text = emoji_pattern.sub(r'', text)
      return text

  # df = pd.read_excel(filepath)
  
  df['review'] = df['review'].apply(lambda x: basic_cleaning(x))
  
  df['review'] = df['review'].apply(lambda x: remove_emoji(x))

  # if uppercase letter exists, add in _ before uppercase letter and replace uppercase letter to lowercase letter 
  df['review'] = df['review'].apply(lambda x: "".join(["_" + ch if ch.isupper() else ch for ch in x]))
  
  df['review'] = df['review'].apply(lambda x: " ".join(slangwords.get(word, word) for word in nltk.word_tokenize(x.lower())))

  # change the character after _ back to uppercase
  df["review"] = df["review"].str.replace(r'_(\w)', lambda m: m.group(1).upper(), regex=True)

  # Remove empty rows in case review only contains emojis
  df['review'].replace('', np.nan, inplace=True)
  df.dropna(subset=['review'], inplace=True)

  # drop rows that have reviews from year 2017and earlier
  df.drop(df[df['date'] < 2017].index, inplace = True)

  df.to_excel('/content/gdrive/MyDrive/data/clean/attractions_reviews_cleaned.xlsx', index=False) 

In [6]:
preprocess_reviews(df)